In [ ]:
import pandas as pd
import numpy as np

# Play with the seed if need be
np.random.seed(42)
num_stocks = 80
industries = ['Tech', 'Healthcare', 'Finance', 'Energy', 'Consumer']

data = {
    'Stock Name': [f'Stock_{i+1}' for i in range(num_stocks)],
    'TSR': np.random.uniform(5, 15, num_stocks),  # TSR between 5% and 15%
    'Grade': np.random.uniform(50, 100, num_stocks),  # Grade between 50 and 100
    'Beta Value': np.random.uniform(0.5, 1.5, num_stocks),  # Beta between 0.5 and 1.5
    'Industry': np.random.choice(industries, num_stocks)  # Randomly assign industry
}

dummy_data = pd.DataFrame(data)

In [ ]:
# load your data 
# something = pd.read_csv('something.csv')

In [2]:
def calculate_grade_score(data, x):
    return (1 - x) * data['TSR'] + x * data['Grade']

In [3]:
# default values set as: 16% for industry allocation and 4.5% mac allocation
# beta value for potfolio should be below 1 
def monte_carlo_simulation(data, max_industry_allocation=0.16, max_stock_allocation=0.045, target_beta=1.0, iterations=10):
    
    best_portfolio = None
    best_portfolio_beta = float('inf')
    
    for _ in range(iterations):
        # The weightage we are giving x will range from 0.45 to 0.6 
        # This will give us cases where our grade is not the main factor and TSR is which provides us with different scores on multiple iterations
        x = np.random.uniform(0.45, 0.60)
        
        # execute the function mention above
        data['Grade Score'] = data.apply(calculate_grade_score, axis=1, args=(x,))
        
        # obtain a weight scross all the investments. Basically getting a ratio so values for the weights will be between 0 and 1 
        # basic normalisation 
        data['Weight'] = data['Grade Score'] / data['Grade Score'].sum()
        
        # Lets focus on each industry
        # what is the current weight allocated to each industry
        data['Industry Allocation'] = data.groupby('Industry')['Weight'].transform('sum')
        data['Allocation_by_restriction'] = np.where(data['Industry Allocation'] > max_industry_allocation, max_industry_allocation, data['Industry Allocation'])
      
        data['Normalised Weight'] = np.where(data['Allocation_by_restriction'] == 0.16, (data['Weight']/data['Industry Allocation'])*0.16, data['Weight'])
        
        # NEED TO ADD WEIGHT FOR THE MAX ALLOCATION

        #Calculate Portfolio Beta
        portfolio_beta = (data['Beta Value'] * data['Normalised Weight']).sum()
        
        if portfolio_beta < target_beta:
            best_portfolio = data[['Stock Name', 'Industry', 'Weight', 'Normalised Weight', 'Beta Value']]
            best_portfolio_beta = portfolio_beta
            break  

    return portfolio_beta, data

In [4]:
x,y = monte_carlo_simulation(dummy_data)

In [5]:
y['Normalised Weight'].sum()

0.7792649974576797

In [6]:
y.groupby('Industry')['Normalised Weight'].sum()

Industry
Consumer      0.160000
Energy        0.160000
Finance       0.139265
Healthcare    0.160000
Tech          0.160000
Name: Normalised Weight, dtype: float64

In [7]:
y['final_weight']  = (y['Normalised Weight']/y['Normalised Weight'].sum())*100

In [8]:
y

,Stock Name,TSR,Grade,Beta Value,Industry,Grade Score,Weight,Industry Allocation,Allocation_by_restriction,Normalised Weight,final_weight
0,Stock_1,8.745401,93.155171,0.867783,Energy,56.798020,0.015260,0.238188,0.160000,0.010251,1.315422
1,Stock_2,14.507143,81.164906,1.132306,Consumer,52.453937,0.014093,0.196056,0.160000,0.011501,1.475873
2,Stock_3,12.319939,66.544901,1.133530,Consumer,43.189013,0.011604,0.196056,0.160000,0.009470,1.215190
3,Stock_4,10.986585,53.177918,1.035775,Finance,35.005179,0.009405,0.139265,0.139265,0.009405,1.206880
4,Stock_5,6.560186,65.549116,0.590290,Energy,40.141282,0.010785,0.238188,0.160000,0.007245,0.929658
...,...,...,...,...,...,...,...,...,...,...,...
75,Stock_76,12.290072,62.102764,1.222452,Consumer,40.647338,0.010921,0.196056,0.160000,0.008912,1.143676
76,Stock_77,12.712703,83.606777,0.780772,Healthcare,53.071135,0.014259,0.258236,0.160000,0.008834,1.133688
77,Stock_78,5.740447,88.080981,0.524316,Energy,52.615095,0.014136,0.238188,0.160000,0.009496,1.218547
78,Stock_79,8.584657,61.881877,1.145472,Healthcare,38.925588,0.010458,0.258236,0.160000,0.006480,0.831516
